In [35]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier

In [3]:
emp = pd.read_excel("employee.xlsx") # Read employee information dataset
perf = pd.read_excel("preformancerating.xlsx") # Read performance rating dataset

emp.head()   # Display first 5 rows of employee data
perf.head()  # Display first 5 rows of performance rating data

,PerformanceID,EmployeeID,ReviewDate,EnvironmentSatisfaction,JobSatisfaction,RelationshipSatisfaction,TrainingOpportunitiesWithinYear,TrainingOpportunitiesTaken,WorkLifeBalance,SelfRating,ManagerRating
0,PR01,79F7-78EC,2013-01-02,5,4,5,1,0,4,4,4
1,PR02,B61E-0F26,2013-01-03,5,4,4,1,3,4,4,3
2,PR03,F5E3-48BB,2013-01-03,3,4,5,3,2,3,5,4
3,PR04,0678-748A,2013-01-04,5,3,2,2,0,2,3,2
4,PR05,541F-3E19,2013-01-04,5,2,3,1,0,4,4,3


In [4]:
df = pd.merge(emp, perf, on="EmployeeID", how="inner") # Merge both datasets using EmployeeID as the key
df.head()  # Show first 5 rows after merging

,EmployeeID,FirstName,LastName,Gender,Age,BusinessTravel,Department,DistanceFromHome (KM),State,Ethnicity,...,PerformanceID,ReviewDate,EnvironmentSatisfaction,JobSatisfaction,RelationshipSatisfaction,TrainingOpportunitiesWithinYear,TrainingOpportunitiesTaken,WorkLifeBalance,SelfRating,ManagerRating
0,3012-1A41,Leonelle,Simco,Female,30,Some Travel,Sales,27,IL,White,...,PR1295,2016-10-30,3,3,2,3,0,4,3,3
1,3012-1A41,Leonelle,Simco,Female,30,Some Travel,Sales,27,IL,White,...,PR1908,2017-10-30,4,4,5,3,1,2,3,2
2,3012-1A41,Leonelle,Simco,Female,30,Some Travel,Sales,27,IL,White,...,PR2617,2018-10-30,5,5,4,3,0,4,5,5
3,3012-1A41,Leonelle,Simco,Female,30,Some Travel,Sales,27,IL,White,...,PR3436,2019-10-30,1,3,2,3,1,3,5,4
4,3012-1A41,Leonelle,Simco,Female,30,Some Travel,Sales,27,IL,White,...,PR411,2014-10-31,3,4,2,1,0,3,4,3


In [5]:
df.info()  # Show data types and non-null counts
df.describe()  # Generate basic descriptive statistics
df.isna().sum()    # Count missing values in each column

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6709 entries, 0 to 6708
Data columns (total 33 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   EmployeeID                       6709 non-null   object        
 1   FirstName                        6709 non-null   object        
 2   LastName                         6709 non-null   object        
 3   Gender                           6709 non-null   object        
 4   Age                              6709 non-null   int64         
 5   BusinessTravel                   6709 non-null   object        
 6   Department                       6709 non-null   object        
 7   DistanceFromHome (KM)            6709 non-null   int64         
 8   State                            6709 non-null   object        
 9   Ethnicity                        6709 non-null   object        
 10  Education                        6709 non-null   int64      

EmployeeID                         0
FirstName                          0
LastName                           0
Gender                             0
Age                                0
BusinessTravel                     0
Department                         0
DistanceFromHome (KM)              0
State                              0
Ethnicity                          0
Education                          0
EducationField                     0
JobRole                            0
MaritalStatus                      0
Salary                             0
StockOptionLevel                   0
OverTime                           0
HireDate                           0
Attrition                          0
YearsAtCompany                     0
YearsInMostRecentRole              0
YearsSinceLastPromotion            0
YearsWithCurrManager               0
PerformanceID                      0
ReviewDate                         0
EnvironmentSatisfaction            0
JobSatisfaction                    0
R

In [6]:
df['Attrition_num'] = df['Attrition'].map({'Yes':1, 'No':0}) # Convert Attrition from Yes/No to 1/0 for analysis

In [7]:
df['Attrition'].value_counts(normalize=True)*100   # Calculate attrition percentage

Attrition
No     66.299001
Yes    33.700999
Name: proportion, dtype: float64

In [8]:
df.groupby("Department")['Attrition'].value_counts(normalize=True)*100  # Attrition percentage by department

Department       Attrition
Human Resources  No           62.046205
                 Yes          37.953795
Sales            No           59.097255
                 Yes          40.902745
Technology       No           70.237256
                 Yes          29.762744
Name: proportion, dtype: float64

In [9]:
df.groupby("JobRole")['Salary'].mean().sort_values(ascending=False)  # Average salary by job role

JobRole
HR Manager                   425598.937500
Analytics Manager            350824.745192
Manager                      331405.835714
HR Business Partner          313027.869565
Engineering Manager          292449.519868
Senior Software Engineer     134137.556680
Machine Learning Engineer    133849.829749
Sales Executive              127258.367521
HR Executive                 103006.591304
Data Scientist                57254.591176
Software Engineer             54347.353073
Sales Representative          41899.194274
Recruiter                     41635.651007
Name: Salary, dtype: float64

In [15]:
df.groupby(pd.cut(df['Age'], bins=[18, 25, 35, 45, 60]), observed=True)['Attrition_num'].mean() # Attrition rate by age groups

Age
(18, 25]    0.506519
(25, 35]    0.352444
(35, 45]    0.119624
(45, 60]    0.099237
Name: Attrition_num, dtype: float64

In [16]:
df[['YearsAtCompany','Attrition_num']].corr()  # Correlation between years at company and attrition

,YearsAtCompany,Attrition_num
YearsAtCompany,1.000000,-0.689653
Attrition_num,-0.689653,1.000000


In [17]:
df.groupby('Gender')['Attrition_num'].mean()  # Attrition rate by gender

Gender
Female               0.320285
Male                 0.367251
Non-Binary           0.307301
Prefer Not To Say    0.000000
Name: Attrition_num, dtype: float64

In [18]:
df[['JobSatisfaction','Attrition_num']].corr()   # Correlation between job satisfaction and attrition

,JobSatisfaction,Attrition_num
JobSatisfaction,1.000000,0.013237
Attrition_num,0.013237,1.000000


In [19]:
df[['WorkLifeBalance','Attrition_num']].corr()   # Correlation between work-life balance and attrition

,WorkLifeBalance,Attrition_num
WorkLifeBalance,1.000000,0.003429
Attrition_num,0.003429,1.000000


In [20]:
df[['SelfRating','ManagerRating']].corr()   # Correlation between self-rating and manager rating

,SelfRating,ManagerRating
SelfRating,1.000000,0.854107
ManagerRating,0.854107,1.000000


In [21]:
df.groupby('JobRole')['JobSatisfaction'].mean().sort_values(ascending=False)  # Average job satisfaction by job role

JobRole
Engineering Manager          3.526490
Recruiter                    3.469799
Data Scientist               3.457353
Machine Learning Engineer    3.453405
Sales Executive              3.435897
Manager                      3.435714
HR Executive                 3.434783
Analytics Manager            3.418269
Software Engineer            3.413043
Sales Representative         3.378323
Senior Software Engineer     3.356275
HR Business Partner          3.347826
HR Manager                   3.250000
Name: JobSatisfaction, dtype: float64

In [22]:
df[['Salary','Attrition_num']].corr()   # Correlation between salary and attrition

,Salary,Attrition_num
Salary,1.000000,-0.211181
Attrition_num,-0.211181,1.000000


In [23]:
df.groupby('Department')['Salary'].mean()  # Average salary by department
df.groupby('JobRole')['Salary'].mean()  # Average salary by job role

JobRole
Analytics Manager            350824.745192
Data Scientist                57254.591176
Engineering Manager          292449.519868
HR Business Partner          313027.869565
HR Executive                 103006.591304
HR Manager                   425598.937500
Machine Learning Engineer    133849.829749
Manager                      331405.835714
Recruiter                     41635.651007
Sales Executive              127258.367521
Sales Representative          41899.194274
Senior Software Engineer     134137.556680
Software Engineer             54347.353073
Name: Salary, dtype: float64

In [24]:
df[['YearsSinceLastPromotion','Attrition_num']].corr()  # Correlation between years since last promotion and attrition

,YearsSinceLastPromotion,Attrition_num
YearsSinceLastPromotion,1.000000,-0.618692
Attrition_num,-0.618692,1.000000


In [25]:
df.groupby('StockOptionLevel')['Attrition_num'].mean()   # Attrition rate by stock option level

StockOptionLevel
0    0.458320
1    0.213434
2    0.180685
3    0.402299
Name: Attrition_num, dtype: float64

In [26]:
df[['TrainingOpportunitiesTaken','ManagerRating']].corr()  # Correlation between training opportunities and manager rating

,TrainingOpportunitiesTaken,ManagerRating
TrainingOpportunitiesTaken,1.000000,0.008496
ManagerRating,0.008496,1.000000


In [27]:
df[['TrainingOpportunitiesTaken','Attrition_num']].corr()  # Correlation between training opportunities and attrition

,TrainingOpportunitiesTaken,Attrition_num
TrainingOpportunitiesTaken,1.000000,-0.007995
Attrition_num,-0.007995,1.000000


In [28]:
df[['DistanceFromHome (KM)','Attrition_num']].corr()  # Correlation between distance from home and attrition

,DistanceFromHome (KM),Attrition_num
DistanceFromHome (KM),1.000000,-0.011707
Attrition_num,-0.011707,1.000000


In [29]:
df.groupby('BusinessTravel')['Attrition_num'].mean()  # Attrition rate based on business travel frequency

BusinessTravel
Frequent Traveller    0.483063
No Travel             0.188216
Some Travel           0.314346
Name: Attrition_num, dtype: float64

In [ ]:
df.to_excel("clean_hr_data.xlsx", index=False)